# Exploratory Data Analysis: Loan Payments

The aim of this project is to conduct exploratory data analysis (EDA) on a database of loan payments for a financial institution.

The code block below initialises the SQL table we will be using to perform this analysis as a Pandas dataframe using the 'db_utils.py' script included in the root directory of this project.

In [ ]:
import db_utils as dbu
import pandas as pd

credentials = dbu.load_yaml('credentials.yaml')
data = dbu.RDSDatabaseConnector(credentials)
data.start_sqlalchemy_engine()
df = data.get_data('loan_payments') # turning SQL table into a Pandas dataframe
pd.set_option('display.max_columns', 50) # SQL table has 43 columns, pandas shows default 9

In [15]:
# Raw dataframe info
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54231 entries, 0 to 54230
Data columns (total 43 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   id                           54231 non-null  int64  
 1   member_id                    54231 non-null  int64  
 2   loan_amount                  54231 non-null  int64  
 3   funded_amount                51224 non-null  float64
 4   funded_amount_inv            54231 non-null  float64
 5   term                         49459 non-null  object 
 6   int_rate                     49062 non-null  float64
 7   instalment                   54231 non-null  float64
 8   grade                        54231 non-null  object 
 9   sub_grade                    54231 non-null  object 
 10  employment_length            52113 non-null  object 
 11  home_ownership               54231 non-null  object 
 12  annual_inc                   54231 non-null  float64
 13  verification_sta

In [14]:
# Raw dataframe sample
df.head(10)

,id,member_id,loan_amount,funded_amount,funded_amount_inv,term,int_rate,instalment,grade,sub_grade,employment_length,home_ownership,annual_inc,verification_status,issue_date,loan_status,payment_plan,purpose,dti,delinq_2yrs,earliest_credit_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_accounts,total_accounts,out_prncp,out_prncp_inv,total_payment,total_payment_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_payment_date,last_payment_amount,next_payment_date,last_credit_pull_date,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type
0,38676116,41461848,8000,8000.0,8000.0,36 months,7.49,248.82,A,A4,5 years,MORTGAGE,46000.0,Not Verified,Jan-2021,Current,n,credit_card,19.54,2,Oct-1987,1,5.0,NaN,12,27,5529.70,5529.70,2982.51,2982.51,2470.30,512.21,0.0,0.0,0.0,Jan-2022,248.82,Feb-2022,Jan-2022,0.0,5.0,1,INDIVIDUAL
1,38656203,41440010,13200,13200.0,13200.0,36 months,6.99,407.52,A,A3,9 years,RENT,50000.0,Not Verified,Jan-2021,Current,n,credit_card,24.20,0,Sep-2001,0,NaN,NaN,15,31,9102.83,9102.83,4885.11,4885.11,4097.17,787.94,0.0,0.0,0.0,Jan-2022,407.52,Feb-2022,Jan-2022,0.0,NaN,1,INDIVIDUAL
2,38656154,41439961,16000,16000.0,16000.0,36 months,7.49,497.63,A,A4,8 years,MORTGAGE,73913.0,Source Verified,Jan-2021,Fully Paid,n,credit_card,16.92,0,Sep-1998,0,69.0,NaN,7,18,0.00,0.00,16824.54,16824.54,16000.00,824.54,0.0,0.0,0.0,Oct-2021,12850.16,None,Oct-2021,0.0,NaN,1,INDIVIDUAL
3,38656128,41439934,15000,15000.0,15000.0,36 months,14.31,514.93,C,C4,1 year,RENT,42000.0,Source Verified,Jan-2021,Fully Paid,n,debt_consolidation,35.52,0,Jun-2008,0,74.0,NaN,6,13,0.00,0.00,15947.47,15947.47,15000.00,947.47,0.0,0.0,0.0,Jun-2021,13899.67,None,Jun-2021,0.0,NaN,1,INDIVIDUAL
4,38656121,41439927,15000,15000.0,15000.0,36 months,6.03,456.54,A,A1,10+ years,MORTGAGE,145000.0,Verified,Jan-2021,Current,n,debt_consolidation,3.33,0,Apr-2002,1,37.0,NaN,23,50,10297.47,10297.47,5473.46,5473.46,4702.53,770.93,0.0,0.0,0.0,Jan-2022,456.54,Feb-2022,Jan-2022,0.0,NaN,1,INDIVIDUAL
5,38656111,41439917,2525,2525.0,2525.0,36 months,11.44,83.20,B,B4,< 1 year,OWN,32000.0,Source Verified,Jan-2021,Current,n,home_improvement,6.60,1,Mar-2011,0,8.0,NaN,3,4,1842.68,1842.68,913.60,913.60,682.32,231.28,0.0,0.0,0.0,Jan-2022,91.39,Feb-2022,Jan-2022,0.0,NaN,1,INDIVIDUAL
6,38656110,41439916,6675,6675.0,6675.0,None,21.99,254.89,E,E5,None,RENT,13536.0,Verified,Jan-2021,Fully Paid,n,debt_consolidation,16.13,0,Nov-2006,2,NaN,NaN,3,4,0.00,0.00,6963.53,6963.53,6675.00,288.53,0.0,0.0,0.0,Mar-2021,6724.95,None,Mar-2021,0.0,NaN,1,INDIVIDUAL
7,38656067,41439872,26500,26500.0,26200.0,None,19.99,701.95,E,E3,< 1 year,RENT,78000.0,Source Verified,Jan-2021,Charged Off,n,debt_consolidation,13.71,0,Mar-2001,0,43.0,NaN,10,37,0.00,0.00,4182.27,4134.92,1197.35,2984.92,0.0,0.0,0.0,Aug-2021,701.95,None,Jan-2022,0.0,43.0,1,INDIVIDUAL
8,38656063,41439868,10000,10000.0,10000.0,60 months,12.99,227.48,C,C2,< 1 year,RENT,50048.0,Source Verified,Jan-2021,Current,n,credit_card,20.67,0,Nov-2005,0,NaN,NaN,8,11,8480.91,8480.91,2722.54,2722.54,1519.09,1203.45,0.0,0.0,0.0,Jan-2022,227.48,Feb-2022,Jan-2022,0.0,NaN,1,INDIVIDUAL
9,38656052,41439857,10000,NaN,10000.0,36 months,8.19,314.25,A,A5,10+ years,MORTGAGE,103000.0,Not Verified,Jan-2021,Current,n,credit_card,15.95,0,Feb-2002,1,35.0,NaN,14,35,6934.63,6934.63,3766.45,3766.45,3065.37,701.08,0.0,0.0,0.0,Jan-2022,314.25,Feb-2022,Jan-2022,0.0,NaN,1,INDIVIDUAL


## Cleaning the data

In this step of the analysis, I'll be cleaning the data by inspecting each column and determining whether I should be changing the data types to better suit analysis, then looking over the columns that are missing data to decide whether or not to impute the missing data.

### Column inspection

In [20]:
inspector = dbu.DataFrameInfo(df)

inspector.print_shape()
print('') # linebreak for readability
inspector.print_nulls()

Shape of the data in dataframe
Number of rows in dataframe: 54231
Number of columns in dataframe: 43


Percentage of nulls for each column in dataframe
id                              0.000000
member_id                       0.000000
loan_amount                     0.000000
funded_amount                   5.544799
funded_amount_inv               0.000000
term                            8.799395
int_rate                        9.531449
instalment                      0.000000
grade                           0.000000
sub_grade                       0.000000
employment_length               3.905515
home_ownership                  0.000000
annual_inc                      0.000000
verification_status             0.000000
issue_date                      0.000000
loan_status                     0.000000
payment_plan                    0.000000
purpose                         0.000000
dti                             0.000000
delinq_2yrs                     0.000000
earliest_credit_line        

### Columns for conversion


From the dataframe sample above, the columns listed below are non-numeric. I will be updating this list of columns with notes as I check each column to determine whether they fit as categories. I will also list the columns that contain date information where the dtype isn't fit for handling datetime data.
1. Potential categories
- term: category OK, null data present, probably better as int in case need to use in calculations later
- grade: category OK
- sub_grade: category OK
- employment_length: category OK, null data present
- home_ownership: category OK, 2 NONEs present, further investigation required
- verification_status: category OK
- loan_status: category OK
- payment_plan: category OK, but 99.9% of data is n, only 1 response y, potential unnecessary column
- purpose: category OK
- application_type: category OK, but unnecessary column since there is only 1 type of response
2. Dates
- issue_date: dt64
- earliest_credit_line: dt64
- last_payment_date: dt64
- next_payment_date: dt64
- last_credit_pull_date: dt64


In [ ]:
object_cols = [
    'term',
    'grade',
    'sub_grade',
    'employment_length',
    'home_ownership',
    'verification_status',
    'loan_status',
    'payment_plan',
    'purpose',
    'application_type'
]

for col in object_cols:
    print(col)
    inspector.count_category(col)
    print('')

#### Anomalies following value counts

Shown below are the rows containing the NONE values from the 'home_ownership' column. 

loan_data_dict.md does not give a list of the types of home ownership, so I was initially questioning whether NONE would represent homeless people or whether it was missing data. The annual income figures for the rows in question are far beyond 50,000 so I will assume that this is simply missing data.

In [24]:
# Further investigation into 'NONE' anomaly in home_ownership column
nones = df.loc[df['home_ownership'] == 'NONE']
nones.head()

,id,member_id,loan_amount,funded_amount,funded_amount_inv,term,int_rate,instalment,grade,sub_grade,employment_length,home_ownership,annual_inc,verification_status,issue_date,loan_status,payment_plan,purpose,dti,delinq_2yrs,earliest_credit_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_accounts,total_accounts,out_prncp,out_prncp_inv,total_payment,total_payment_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_payment_date,last_payment_amount,next_payment_date,last_credit_pull_date,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type
53947,179929,179912,15000,15000.0,3455.522875,36 months,15.65,524.77,F,F3,6 years,NONE,72000.0,Not Verified,Dec-2013,Does not meet the credit policy. Status:Charge...,n,other,8.73,2,May-1996,3,17.0,0.0,7,26,0.0,0.0,1694.31,397.67,662.04,386.64,0.0,645.63,6.38,Feb-2014,524.77,Dec-2014,Jan-2015,0.0,NaN,1,INDIVIDUAL
54171,121535,121373,2800,2800.0,1625.000000,36 months,8.70,88.65,B,B1,< 1 year,NONE,120000.0,Not Verified,Aug-2013,Fully Paid,n,other,15.00,0,May-1999,1,NaN,NaN,6,8,0.0,0.0,3191.12,1851.97,2800.00,391.12,0.0,0.00,0.00,Sep-2016,89.80,None,Aug-2013,NaN,NaN,1,INDIVIDUAL


#### Converting the columns

In [ ]:
# Converting categorical data columns from object to category data type
category_columns = ["grade", "sub_grade", "employment_length", "home_ownership", "verification_status", "loan_status", "payment_plan", "purpose", "application_type"]
convert_to_categories = dbu.DataTransform(df, category_columns)
df = convert_to_categories.to_category()

In [ ]:
# Converting columns containing dates from object(month, year) to datetime64(yyyy-mm-dd) data type
date_columns = ["issue_date", "earliest_credit_line", "last_payment_date", "next_payment_date", "last_credit_pull_date"]
convert_to_datetime = dbu.DataTransform(df, date_columns)
df = convert_to_datetime.to_datetime()

The term column makes more sense as an int data type as every data point in this column is in "number, months" format. In order to convert this to an int column, I'll be editing the strings and renaming the column to clarify the information the string already tells us: the numbers represent an amount of months.

In [ ]:
# Renaming term column to prevent confusion later
df = df.rename(columns={"term": "term_in_months"})

# Removing "months" from each row
df["term_in_months"] = df["term_in_months"].str.replace(" months", "")

In [ ]:
# Converting term column from object to int, decided int is a better fit than category as all data points are numeric
term = ["term_in_months"]
convert_to_int = dbu.DataTransform(df, "term_in_months")
df = convert_to_int.to_int()

In [ ]:
# Dataframe info post-conversion
df.info()

In [ ]:
# Dataframe sample post-conversion
df.head(10)

### Columns with null data 

An additional column with null data was identified through column inspection above (home_ownership contains 2 NONEs, which register as a value rather than a null), but these are not counted as nulls by pandas and thus will not show in the following plot and null counts, but I will be including it in the null_cols variable anyway as a reminder.

In [ ]:
# Visualising columns with null data
import missingno as msno

plot = dbu.Plotter(df)

# Filtering data so that only columns with nulls are included in visualisation
data_filter = msno.nullity_filter(df, filter='bottom', p=0.9999, n=50)

plot.plot_nulls(filter=data_filter) # in the plot, null information is represented by white space

In [23]:
null_cols = [
    'funded_amount', 
    'term', 
    'int_rate', 
    'employment_length', 
    'home_ownership',
    'mths_since_last_delinq', 
    'mths_since_last_record', 
    'last_payment_date', 
    'next_payment_date', 
    'last_credit_pull_date', 
    'collections_12_mths_ex_med', 
    'mths_since_last_major_derog'
    ]

for col in null_cols:
    print(col)
    inspector.count_nulls(col)
    print(' ')

funded_amount
Percentage of non-null data in funded_amount: 94.455
Percentage of null data in funded_amount: 5.545
 
term
Percentage of non-null data in term: 91.201
Percentage of null data in term: 8.799
 
int_rate
Percentage of non-null data in int_rate: 90.469
Percentage of null data in int_rate: 9.531
 
employment_length
Percentage of non-null data in employment_length: 96.094
Percentage of null data in employment_length: 3.906
 
home_ownership
Percentage of non-null data in home_ownership: 100.0
Percentage of null data in home_ownership: 0.0
 
mths_since_last_delinq
Percentage of non-null data in mths_since_last_delinq: 42.833
Percentage of null data in mths_since_last_delinq: 57.167
 
mths_since_last_record
Percentage of non-null data in mths_since_last_record: 11.398
Percentage of null data in mths_since_last_record: 88.602
 
last_payment_date
Percentage of non-null data in last_payment_date: 99.865
Percentage of null data in last_payment_date: 0.135
 
next_payment_date
Percenta

### Columns to consider dropping
- payment_plan: 99.9% of data is n, only 1 response y, potential unnecessary column
- application_type: unnecessary column since there is only 1 type of response

In [ ]:
# Dropping unnecessary columns
df = df.drop(columns=["payment_plan", "application_type"])

In [ ]:
# Dataframe sample with columns dropped
df.head(10)

### Imputation